In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
file = 'datasetTC4.dat'
data = pd.read_csv(file, header=None, sep=' ')
data = data.iloc[:, :-1]
data.head()

,0,1,2,3,4,5
0,63.03,22.55,39.61,40.48,98.67,-0.25
1,39.06,10.06,25.02,29.00,114.41,4.56
2,68.83,22.22,50.09,46.61,105.99,-3.53
3,69.30,24.65,44.31,44.64,101.87,11.21
4,49.71,9.65,28.32,40.06,108.17,7.92


In [3]:
# Passo 1 - Normalizar os dados
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized

array([[ 0.14722652,  0.50111133, -0.66512805, -0.18460234, -1.44783071,
        -0.70794606],
       [-1.24570706, -0.74889057, -1.45276272, -1.04124965, -0.26402779,
        -0.57967342],
       [ 0.48427345,  0.46808485, -0.0993699 ,  0.27282344, -0.89729467,
        -0.79541679],
       ...,
       [ 0.05541029,  0.51512256, -0.31098936, -0.31369641,  0.58283504,
        -0.77354911],
       [-0.88599664, -0.88600047, -0.55877847, -0.47711606,  0.04734096,
        -0.69567882],
       [-1.54904929, -1.24829085, -0.82546218, -1.05841244,  0.45347411,
        -0.70661266]])

## Questão Única: Implementar os algoritmos K-médias e K-medianas usando o conjunto de dados disponibilizado no SIGAA (datasetTC4.dat).

### Usando o algoritmo K-médias, estimar o número de agrupamentos através dos ı́ndices de validação:

In [4]:
# Função para executar o algoritmo K-médias com várias rodadas e retornar a melhor
def kmeans_with_min_ssd(X, k, n_runs=10):
    min_ssd = np.inf
    best_labels = None
    best_centroids = None

    for _ in range(n_runs):
        kmeans = KMeans(n_clusters=k, random_state=np.random.randint(1000))
        labels = kmeans.fit_predict(X)
        ssd = np.sum((X - kmeans.cluster_centers_[labels])**2)

        if ssd < min_ssd:
            min_ssd = ssd
            best_labels = labels
            best_centroids = kmeans.cluster_centers_

    return best_labels, best_centroids

## 1 - Dunn

In [5]:
# Função para calcular o índice de Dunn
def dunn_index(X, labels, centroids):
    X_array = X.values if isinstance(X, pd.DataFrame) else X
    dist_matrix = np.linalg.norm(X_array - centroids[labels], axis=-1)

    min_diameter = np.inf
    for k in range(len(centroids)):
        cluster_points = X_array[labels == k]
        cluster_diameter = np.max(np.linalg.norm(cluster_points - centroids[k], axis=-1))
        min_diameter = min(min_diameter, cluster_diameter)

    mask = labels != labels[:, np.newaxis]
    min_intercluster_distance = np.min(dist_matrix[mask.any(axis=1)])

    if min_diameter == 0:
        return np.nan
    else:
        return min_intercluster_distance / min_diameter

In [6]:
# Passo 2 e 3 - Calcular o índice de Dunn para diferentes valores de K
Kmax = 100
dunn_scores = []

for k in range(2, Kmax + 1):
    labels, centroids = kmeans_with_min_ssd(data_normalized, k)
    dunn_scores.append(dunn_index(data_normalized, labels, centroids))

# Passo 4 - Escolher o valor ótimo Kopt como aquele que otimiza o índice de Dunn
optimal_k = np.argmax(dunn_scores) + 2  # Adiciona 2 porque começamos com K=2
optimal_labels, optimal_centroids = kmeans_with_min_ssd(data_normalized, optimal_k)

# Passo 5 - Particionar os dados entre os Kopt agrupamentos
optimal_labels, _ = kmeans_with_min_ssd(data_normalized, optimal_k)
data['Cluster'] = optimal_labels

# Reportar estatísticas descritivas dos atributos por agrupamento
#grouped_stats = data.groupby('Cluster').describe().transpose()

# Mostrar resultados
print("Índice de Dunn para diferentes valores de K:", dunn_scores)
print("Valor ótimo de K (Kopt):", optimal_k)
#print("\nEstatísticas descritivas por agrupamento:")
#print(grouped_stats)

Índice de Dunn para diferentes valores de K: [0.11159227241921335, 0.05493040620388903, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Valor ótimo de K (Kopt): 4


## Davies-Bouldin

In [8]:
# Função para calcular o índice de Davies-Bouldin
def davies_bouldin_index(X, labels, centroids):
    X_array = X.values if isinstance(X, pd.DataFrame) else X
    num_clusters = len(centroids)
    distances_matrix = np.zeros((num_clusters, num_clusters))

    for i in range(num_clusters):
        for j in range(i + 1, num_clusters):
            distances_matrix[i, j] = np.linalg.norm(centroids[i] - centroids[j])
            distances_matrix[j, i] = distances_matrix[i, j]

    max_cluster_distances = np.zeros(num_clusters)

    for k in range(num_clusters):
        cluster_points = X_array[labels == k]
        mean_distance = np.mean(np.linalg.norm(cluster_points - centroids[k], axis=-1))
        max_cluster_distances[k] = max([distances_matrix[k, j] for j in range(num_clusters) if j != k])

    return np.sum(max_cluster_distances) / num_clusters

In [9]:
# Passo 2 e 3 - Calcular o índice de Davies-Bouldin para diferentes valores de K
Kmax = 100
davies_bouldin_scores = []

for k in range(2, Kmax + 1):
    labels, centroids = kmeans_with_min_ssd(data_normalized, k)
    davies_bouldin_scores.append(davies_bouldin_index(data_normalized, labels, centroids))

# Passo 4 - Escolher o valor ótimo Kopt como aquele que minimiza o índice de Davies-Bouldin
optimal_k = np.argmin(davies_bouldin_scores) + 2  # Adiciona 2 porque começamos com K=2
optimal_labels, optimal_centroids = kmeans_with_min_ssd(data_normalized, optimal_k)

# Passo 5 - Particionar os dados entre os Kopt agrupamentos
optimal_labels, _ = kmeans_with_min_ssd(data_normalized, optimal_k)
data['Cluster'] = optimal_labels

# Mostrar resultados
print("Índice de Davies-Bouldin para diferentes valores de K:", davies_bouldin_scores)
print("Valor ótimo de K (Kopt):", optimal_k)

Índice de Davies-Bouldin para diferentes valores de K: [3.077053165076801, 3.6865830561429807, 12.845775800170413, 12.674569845800423, 12.729995027147107, 12.834111075415828, 12.76569146814768, 12.692522187139486, 12.70673346579874, 12.72624608296034, 12.63766554011689, 12.771871230549436, 12.702428807286369, 12.762860373066902, 12.596607979320957, 12.629628563110675, 12.589634989043232, 12.684696559663957, 12.672351558764621, 12.626947633589262, 12.583141004526414, 12.723143469747582, 12.680591186046422, 12.660966322884414, 12.702235146933628, 12.722535278471925, 12.59956575128233, 12.621722036209539, 12.643162090965747, 12.654928228857111, 12.636882881709269, 12.68242184879098, 12.623625623411014, 12.554023596486246, 12.615782271010206, 12.699845124524888, 12.555684158834183, 12.640960204271112, 12.621254234894693, 12.693645002685807, 12.621378840808735, 12.735900494198729, 12.652382504314282, 12.62439397694871, 12.703748764343365, 12.523387775468827, 12.555774712050697, 12.617586471

## Calinski-Harabasz

In [10]:
# Função para calcular a métrica de Calinski-Harabasz
def calinski_harabasz_index(X, labels, centroids):
    X_array = X.values if isinstance(X, pd.DataFrame) else X
    num_samples, num_features = X_array.shape
    num_clusters = len(centroids)

    overall_mean = np.mean(X_array, axis=0)
    overall_ss = np.sum(np.sum((X_array - overall_mean) ** 2, axis=1))

    between_cluster_ss = 0
    within_cluster_ss = 0

    for k in range(num_clusters):
        cluster_points = X_array[labels == k]
        cluster_size = len(cluster_points)

        cluster_mean = np.mean(cluster_points, axis=0)
        between_cluster_ss += cluster_size * np.sum((cluster_mean - overall_mean) ** 2)

        within_cluster_ss += np.sum(np.sum((cluster_points - cluster_mean) ** 2, axis=1))

    return (between_cluster_ss / (num_clusters - 1)) / (within_cluster_ss / (num_samples - num_clusters))

In [11]:
# Passo 2 e 3 - Calcular a métrica de Calinski-Harabasz para diferentes valores de K
Kmax = 100
calinski_harabasz_scores = []

for k in range(2, Kmax + 1):
    labels, centroids = kmeans_with_min_ssd(data_normalized, k)
    calinski_harabasz_scores.append(calinski_harabasz_index(data_normalized, labels, centroids))

# Passo 4 - Escolher o valor ótimo Kopt como aquele que otimiza a métrica de Calinski-Harabasz
optimal_k = np.argmax(calinski_harabasz_scores) + 2  # Adiciona 2 porque começamos com K=2
optimal_labels, optimal_centroids = kmeans_with_min_ssd(data_normalized, optimal_k)

# Passo 5 - Particionar os dados entre os Kopt agrupamentos
optimal_labels, _ = kmeans_with_min_ssd(data_normalized, optimal_k)
data['Cluster'] = optimal_labels

# Mostrar resultados
print("Índice de Calinski-Harabasz para diferentes valores de K:", calinski_harabasz_scores)
print("Valor ótimo de K (Kopt):", optimal_k)

Índice de Calinski-Harabasz para diferentes valores de K: [189.336302284451, 153.5759520609528, 134.4270301642051, 124.51797276477636, 118.68010755134631, 112.66011808047709, 107.0869638294243, 103.16330859883703, 98.94893068819539, 94.36629483131087, 91.17758208725337, 88.03189781765347, 86.45688902550786, 83.65504757851949, 81.72055756165052, 80.2384286735316, 78.86470376825687, 76.14949051683126, 74.91389179641587, 74.55068414902931, 72.34337025807213, 71.43813502898867, 70.4923989370308, 69.9224144972078, 68.678715282557, 67.80753842533146, 66.59432674543773, 66.88546614095844, 66.03810723726387, 64.23476553457839, 63.87679672967513, 62.906627671827835, 63.12799684903986, 62.52171366814858, 60.892913377195974, 61.91548353924671, 59.53503345890483, 59.58119123978232, 59.184054686467725, 58.17224766256927, 57.88117604477126, 56.9608620188561, 57.005862712567854, 56.33213256775073, 56.9925765768663, 55.92514120707815, 55.68258981207668, 55.84448565608198, 55.18738993882095, 54.3216569

## (i) Qual valor para o número de agrupamentos foi sugerido por cada técnica de validação?

## (ii) Se houve divergência entre os resultados sugeridos pelos ı́ndices, o que justifica tal divergência?